In [4]:
my_api_key = "b141795955348002155a81415a728a61faab9746"
url = "https://api.census.gov/data/{}/{}?get={}&for={}&in={}&key={}"

import pandas as pd
import requests
from tqdm import tqdm
tqdm.pandas(desc="Validating")


district_test = ('congressional%20district:*', 'state:11', my_api_key)
tract_test = ('tract:000100', 'state:11%20county:001', my_api_key)
district_full = ('congressional%20district:*', 'state:*', my_api_key)
tract_full = ('tract:*', 'state:*%20county:*', my_api_key)
              

level_test = {'district':district_test, 'tract':tract_test}
level_full = {'district':district_full, 'tract':tract_full}

tup_wrap = lambda x: tuple(x)

url = "https://api.census.gov/data/{}/{}?get={}&for={}&in={}&key={}"
def build_tests(df):
    return pd.Series(pd.Series(list(zip(df.year.astype(str),df.api, df.field)))
                     +df.level.map(level_test)).apply(lambda x: url.format(*x))
    

def test_fields(df):
    return build_tests(df).progress_apply(requests.get).progress_apply(lambda x: x.ok)


In [7]:
# dfq

apis = ['acs1', 'acs1', 'acs5', 'acs5', 'sf1', 'sf1', 'sf3', 'sf3']
years = [2015, 2015, 2015, 2015, 2010, 2010, 2000, 2000]
fields = ['B19001B_004E','B19001B_004E','B19001B_004E','B19001B_004E','B19001B_004E','B19001B_004E','B19001B_004E','B19001B_004E']
levels = ['district', 'tract','district', 'tract','district', 'tract','district', 'tract']

df = pd.DataFrame()
df['api'] = apis
df['year'] = years
df['field'] = fields
df['level'] = levels
df

,api,year,field,level
0,acs1,2015,B19001B_004E,district
1,acs1,2015,B19001B_004E,tract
2,acs5,2015,B19001B_004E,district
3,acs5,2015,B19001B_004E,tract
4,sf1,2010,B19001B_004E,district
5,sf1,2010,B19001B_004E,tract
6,sf3,2000,B19001B_004E,district
7,sf3,2000,B19001B_004E,tract


In [8]:
df['valid'] = test_fields(df)
df

Validating: 100%|██████████████████████████████████████████████████████████████████████| 8/8 [00:00<00:00, 8002.49it/s]


,api,year,field,level,valid
0,acs1,2015,B19001B_004E,district,True
1,acs1,2015,B19001B_004E,tract,False
2,acs5,2015,B19001B_004E,district,True
3,acs5,2015,B19001B_004E,tract,True
4,sf1,2010,B19001B_004E,district,False
5,sf1,2010,B19001B_004E,tract,False
6,sf3,2000,B19001B_004E,district,False
7,sf3,2000,B19001B_004E,tract,False
